# 단일 관측 자료에 여러 모델 자료 병합

<h2>파일 구조</h2>

<pre>
<strong>관측 자료</strong>
관측 자료 경로 = Y:\DATA\2023_SC2_ROAD_OBS\DATA_OBS\by_sensor
관측 파일 이름 규칙
{site}_{번호}_{road_name}_{sensor_name}_{year+monrh+day}_{road_direction}.csv

<strong>모델 자료</strong>
모델 자료 경로 = Y:\DATA\2023_SC2_ROAD_OBS\MODEL
모델 자료 경료 이름 규칙
{모델자료 경로}/{site}/{model}/1.OUT/{year+month}/{day:02f}
모델 파일 이름 규칙
KMA_HUFS_ROAD-P_{site}_{year+month+day+hour+minute}.csv

*site = {seoul / siheung}
*model = {jr, mg, org, ss, yc}
</pre>
<hr />
<h2>코드 흐름</h2>
<pre>
1. 관측 자료 불러오기 : 시간 / 도로 / 도로 방향 특정하기
2. 특정된 정보를 통해서 모델 자료 불러오기
3. 불러온 모델 자료의 컬럼명 변경 {modle}_{관측변수이름}
4. 모델 자료 전체 위경도를 통한 통합
5. 통합된 모델 자료와 관측 자료 병합
6. RMSE를 통한 모델 자료 평가/검증
</pre>

### 필요 패키지 로드

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn.neighbors import KDTree

import os
import glob

c:\Users\user\anaconda3\envs\DT2023\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 관측 자료 불러오기

In [61]:
# 사용할 관측 데이터

# 디렉토리 내의 모든 CSV 파일 목록 가져오기
csv_files = glob.glob('./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/' + '*.csv')
csv_files = csv_files[:-4]
# 결과 출력
print("CSV 파일 목록:")
for file in csv_files:

    print(file)
    obs_df = pd.read_csv(file, low_memory=False)
    obs_df.columns = obs_df.columns.str.lower()


    file_split = file.split('\\')
    csv_name = file_split[-1]

    file_info = csv_name.split('_')
    site = file_info[0]
    road_name = file_info[-4]
    sensor = file_info[-3]
    date = file_info[-2]
    road_dir_1 = file_info[-1]
    road_dir_1 = road_dir_1.split('.')[0]

    print(site, road_name, sensor, date, road_dir_1)
    DAY = date[6:8]
    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'])
    hours = obs_df['timestamp'].dt.hour.unique()
    hours = list(hours)



    model_fir_dir = "./DATA/MODEL/"
    models = {'jr', 'mg', 'org', 'ss', 'yc'}
    models_df = pd.DataFrame()

    for model in models:
        yyyymm = date[:6]
        day = date[6:8]
        day_int = int(day)
        days = [day_int - 1, day_int]
        model_1_df = pd.DataFrame()
        for day in days:
            model_pattern = f"{model_fir_dir}/{model}/{yyyymm}/{day:02d}/*.csv"
            
            # 해당 패턴에 맞는 모든 CSV 파일 찾기
            csv_files = glob.glob(model_pattern)
            print(csv_files)
            if not csv_files:
                print(f"해당 조건의 {model} 모델 파일이 경로에 존재하지 않습니다")
            else:
                model_df = pd.read_csv(csv_files[0], low_memory=False)
                
                # Rename the columns
                model_df = model_df.rename(columns={
                    'road_temp': f'road_temp_{model}',
                    'road_hydro': f'road_hydro_{model}',
                    'road_ice': f'road_ice_{model}',
                })
                model_df["date_time"] = pd.to_datetime(model_df["date_time"])
                model_df["year"] = model_df["date_time"].dt.year
                model_df["month"] = model_df["date_time"].dt.month
                model_df["day"] = model_df["date_time"].dt.day
                model_df["hour"] = model_df["date_time"].dt.hour
                model_df = model_df.drop(['date_time','update_time','altitude','link_id'], axis=1)
                # Define columns for merging (adjust based on your data)
                model_1_df =  pd.concat([model_1_df,model_df])

                cond1 = model_1_df['road_name'] == road_name
                cond2 = model_1_df['direction'].str.startswith(road_dir_1)
                cond3 = model_1_df['hour'].isin(hours)
                cond4 = model_1_df['day']==int(DAY)
                
                model_1_df = model_1_df[cond1&cond2&cond3&cond4]
        merge_columns = ['loc', 'lon', 'lat', 'seq', 'p_hour','road_name','direction', 'year','month','day','hour']
                
        # Merge DataFrames
        if models_df.empty:
            models_df = model_1_df
        else:
            models_df = pd.merge(models_df, model_1_df, on=merge_columns)

    models_df = models_df.drop_duplicates(subset=['lon','lat','year','month','day','hour','p_hour'])
    data_obs_test = pd.read_csv(file, low_memory=False)

    #KDTree 방법
    road_df = pd.DataFrame()
    # 첫 번째 줄의 위경도 데이터
    observ_line = np.array(list(zip(data_obs_test['longitude'], data_obs_test['latitude'])))

    # 두 번째 줄의 위경도 데이터
    model_line = np.array(list(zip(models_df['lon'], models_df['lat'])))

    # KDTree 객체 생성
    tree = KDTree(observ_line)

    # 각 점마다 가장 가까운 점을 찾아 매칭
    matched_points = []
    for point in model_line:
        _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
        matched_points.append(observ_line[index[0]])

    matched_lon = [point[0][0] for point in matched_points]
    matched_lat = [point[0][1] for point in matched_points]

    df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1], 'longitude': matched_lon, 'latitude': matched_lat})

    days = models_df['day'].unique()

    for day in days:

        model_date_2 = models_df[models_df['day'] == day]

        total_df = pd.merge(df,data_obs_test,on=['longitude','latitude'])
        total_df = pd.merge(total_df,model_date_2,on=['lon','lat'])
        total_df = total_df.drop_duplicates()
        road_df = pd.concat([road_df,total_df])

    road_df.to_csv(f'{road_name}{road_dir_1}{DAY}.csv', index=False)



CSV 파일 목록:
./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료\seoul_1_gangbyeon_vaisala_20230119_U.csv
seoul gangbyeon vaisala 20230119 U
['./DATA/MODEL//ss/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//ss/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//mg/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//mg/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//jr/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//jr/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//yc/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//yc/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//org/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//org/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료\seoul_2_naebu_vaisala_20230119_U.csv
seoul naebu vaisala 20230119 U
['./DATA/MODEL//ss/202301/18\\KMA_HUFS_

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [3]:
# ver2

# 사용할 관측 데이터

# 디렉토리 내의 모든 CSV 파일 목록 가져오기
csv_files = glob.glob('./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료/' + '*.csv')
print("CSV 파일 목록:")
for file in csv_files:

    print(file)
    obs_df = pd.read_csv(file, low_memory=False)
    obs_df.columns = obs_df.columns.str.lower()

    file_info = file.split('_')

    road_name = file_info[-4]
    date = file_info[-2]
    road_dir_1 = file_info[-1]
    road_dir_1 = road_dir_1.split('.')[0]
    date = file_info[-2]
    print(road_name, date, road_dir_1)

    obs_df['timestamp'] = pd.to_datetime(obs_df['timestamp'])
    hours = obs_df['timestamp'].dt.hour.unique()
    obs_df["day"] = obs_df["timestamp"].dt.day
    obs_df["hour"] = obs_df["timestamp"].dt.hour
    obs_df["ddhh"] = obs_df["day"] * 100 + obs_df["hour"]

    ddhh = obs_df['ddhh'].unique()
    ddhh = list(ddhh)


    model_fir_dir = "./DATA/MODEL/"
    models = {'jr', 'mg', 'org', 'ss', 'yc'}
    models_df = pd.DataFrame()

    for model in models:
        yyyymm = date[:6]
        day = date[6:8]
        day_int = int(day)
        days = [day_int - 1, day_int]
        model_1_df = pd.DataFrame()
        for day in days:
            model_pattern = f"{model_fir_dir}/{model}/{yyyymm}/{day:02d}/*.csv"
            
            # 해당 패턴에 맞는 모든 CSV 파일 찾기
            csv_files = glob.glob(model_pattern)
            print(csv_files)
            if not csv_files:
                print(f"해당 조건의 {model} 모델 파일이 경로에 존재하지 않습니다")
            else:
                model_df = pd.read_csv(csv_files[0], low_memory=False)
                
                # Rename the columns
                model_df = model_df.rename(columns={
                    'road_temp': f'road_temp_{model}',
                    'road_hydro': f'road_hydro_{model}',
                    'road_ice': f'road_ice_{model}',
                })
                model_df["date_time"] = pd.to_datetime(model_df["date_time"])
                model_df["day"] = model_df["date_time"].dt.day
                model_df["hour"] = model_df["date_time"].dt.hour
                model_df["ddhh"] = model_df["day"]*100+ model_df["hour"]
                model_df = model_df.drop(['update_time','date_time','altitude','link_id','seq', 'p_hour'], axis=1)
                # Define columns for merging (adjust based on your data)
                
                cond1 = model_df['road_name'] == road_name
                cond2 = model_df['direction'].str.startswith(road_dir_1)
                cond3 = model_df['ddhh'].isin(ddhh)
                
                model_df1 = model_df[cond1&cond2&cond3]
                model_1_df =  pd.concat([model_1_df,model_df1])

        merge_columns = ['loc', 'lon', 'lat', 'road_name','direction', 'day','hour','ddhh']
                
        # Merge DataFrames
        if models_df.empty:
            models_df = model_1_df
        else:
            models_df = pd.merge(models_df, model_1_df, on=merge_columns)

    #models_df = models_df.drop_duplicates(subset=['lon','lat','day','hour'])
            
    data_obs_test = pd.read_csv(file, low_memory=False)

    #KDTree 방법
    road_df = pd.DataFrame()
    # 첫 번째 줄의 위경도 데이터
    observ_line = np.array(list(zip(data_obs_test['longitude'], data_obs_test['latitude'])))

    # 두 번째 줄의 위경도 데이터
    model_line = np.array(list(zip(models_df['lon'], models_df['lat'])))

    # KDTree 객체 생성
    tree = KDTree(observ_line)

    # 각 점마다 가장 가까운 점을 찾아 매칭
    matched_points = []
    for point in model_line:
        _, index = tree.query([point], k=1)  # k=1로 설정하여 가장 가까운 점 하나만 선택
        matched_points.append(observ_line[index[0]])

    matched_lon = [point[0][0] for point in matched_points]
    matched_lat = [point[0][1] for point in matched_points]

    df = pd.DataFrame({'lon': model_line[:, 0], 'lat': model_line[:, 1], 'longitude': matched_lon, 'latitude': matched_lat})

    days = models_df['day'].unique()

    for day in days:

        model_date_2 = models_df[models_df['day'] == day]

        total_df = pd.merge(df,data_obs_test,on=['longitude','latitude'])
        total_df = pd.merge(total_df,model_date_2,on=['lon','lat','hour','day'])
        total_df = total_df.drop_duplicates()
        road_df = pd.concat([road_df,total_df])

    road_df.to_csv(f'{road_name}{road_dir_1}{date}.csv', index=False)


CSV 파일 목록:
./DATA/20230705_스시2_서울&시흥_1월관측_후처리자료\seoul_1_gangbyeon_vaisala_20230119_U.csv
gangbyeon 20230119 U
['./DATA/MODEL//jr/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//jr/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//org/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//org/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//ss/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//ss/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//mg/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//mg/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']
['./DATA/MODEL//yc/202301/18\\KMA_HUFS_ROAD-P_seoul_20230118150000.csv']
['./DATA/MODEL//yc/202301/19\\KMA_HUFS_ROAD-P_seoul_20230119150000.csv']


KeyError: 'hour'

In [5]:
model_date_2

,loc,lon,lat,road_name,direction,road_temp_jr,road_hydro_jr,road_ice_jr,day,hour,...,road_ice_org,road_temp_ss,road_hydro_ss,road_ice_ss,road_temp_mg,road_hydro_mg,road_ice_mg,road_temp_yc,road_hydro_yc,road_ice_yc
0,seoul,127.113670,37.556764,gangbyeon,URL,11.994510,0,0,19,15,...,0,7.787102,0,0,14.515960,0,0,8.450951,0,0
1,seoul,127.113670,37.556764,gangbyeon,URL,11.994510,0,0,19,15,...,0,7.787102,0,0,14.515960,0,0,9.555754,0,0
2,seoul,127.113670,37.556764,gangbyeon,URL,11.994510,0,0,19,15,...,0,7.787102,0,0,15.717401,0,0,8.450951,0,0
3,seoul,127.113670,37.556764,gangbyeon,URL,11.994510,0,0,19,15,...,0,7.787102,0,0,15.717401,0,0,9.555754,0,0
4,seoul,127.113670,37.556764,gangbyeon,URL,11.994510,0,0,19,15,...,0,8.832914,0,0,14.515960,0,0,8.450951,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112379,seoul,126.863162,37.576455,gangbyeon,URL,12.900761,0,0,19,15,...,0,7.762387,0,0,15.693575,0,0,9.530671,0,0
112380,seoul,126.863162,37.576455,gangbyeon,URL,12.900761,0,0,19,15,...,0,8.807510,0,0,14.495719,0,0,8.427282,0,0
112381,seoul,126.863162,37.576455,gangbyeon,URL,12.900761,0,0,19,15,...,0,8.807510,0,0,14.495719,0,0,9.530671,0,0
112382,seoul,126.863162,37.576455,gangbyeon,URL,12.900761,0,0,19,15,...,0,8.807510,0,0,15.693575,0,0,8.427282,0,0
